## 2. 동형이의어 구별 (BoolQ)

- 전체적인 코드의 뼈대 / 하이퍼 파라미터는 같은 조의 BoolQ의 코드를 사용함
- 사용모델 : KLUE-roberta-large
- Epoch: 10
- warm up: 10% training step
- Learning rate: 8e-6
- Batch size: 4 (colab에서 4를 넘어가면 memory 초과)
- Classifier: 1 layer linear model ( layer/activaation을 추가해도 효과 미미)

## [동형이의어구별] 안형서 (2017-12932)
 - 같은 조의 김장현님 코드를 뼈대로 하여 수정

In [1]:

#!pip install transformers  #if colab !!!!!!!!
#cur_dir="./drive/MyDrive/NLP/final/"   #if colab !!!!!!!!

#cur_dir="../" # if local!!!!!!!!!!!!!!1
cur_dir="./" # if local!!!!!!!!!!!!!!1

In [2]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import transformers
import pandas
from transformers import AutoTokenizer
from transformers import RobertaModel, RobertaConfig ,RobertaPreTrainedModel
from transformers import AdamW
import time
import argparse
from tqdm.auto import tqdm
import numpy as np
import random

transformers.logging.set_verbosity(40) # Turn off warning

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [4]:
ensen_num=6

save_dir = cur_dir+'result_wic'

if not os.path.exists( save_dir):
    os.makedirs(save_dir)


In [5]:
"""
seed=5555

torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

"""

"\nseed=5555\n\ntorch.manual_seed(seed)\ntorch.backends.cudnn.deterministic = True\ntorch.backends.cudnn.benchmark = False\nnp.random.seed(seed)\nrandom.seed(seed)\nos.environ['PYTHONHASHSEED'] = str(seed)\nif torch.cuda.is_available():\n    torch.cuda.manual_seed(seed)\n    torch.cuda.manual_seed_all(seed)\n\n"

### Load QA data
- Train, Dev 데이터가 base_path에 들어있어야 합니다. (default: './data')

# 데이터셋 만들기
 - 그 토큰이 명확하게 쪼개져야 하기때문에 앞뒤로 special token을 삽입해 줌으로써 그 토큰이 앞이나 뒤에서 다른 글자와 합쳐지지 않고 명확하게 쪼개지게 해준다.
 - 또한, 모델의 끝부분에서 그 토큰의 마지막 layer의 출력 값을 입력으로 쓰고 나머지 토큰들의 결과는 안쓰니, 그 부분만 1, 나머지는 0 인 mask가 필요하다. 이를 위해 만든 speical token을 이용하여 mask를 만들어준다.
 - 시도해본 것 1. 토큰의 마지막 부분만 1만들기. 2. .쪼개진 토큰 전부 1로 만들기

In [6]:
def load_data(path, tokenizer):
    dataset = pandas.read_csv(path,
                              delimiter='\t',
                              names=['ID', 'Target', 'text1', 'text2','answer','s_s1','e_s1','s_s2','e_s2'],
                              header=0)

    dataset_text1=[]
    dataset_text2=[]

    for i,text1 in enumerate(dataset['text1']):
        text= text1[:dataset["s_s1"][i]]+"[WORD1S]"+text1[dataset["s_s1"][i]:dataset["e_s1"][i]]+"[WORD1E]"+text1[dataset["e_s1"][i]:]
        dataset_text1.append(text)

    for i,text2 in enumerate(dataset['text2']):
        text= text2[:dataset["s_s2"][i]]+"[WORD2S]"+text2[dataset["s_s2"][i]:dataset["e_s2"][i]]+"[WORD2E]"+text2[dataset["e_s2"][i]:]
        dataset_text2.append(text)
    
    word1s_tok_idx=tokenizer.encode("[WORD1S]")[1]
    word1e_tok_idx=tokenizer.encode("[WORD1E]")[1]
    word2s_tok_idx=tokenizer.encode("[WORD2S]")[1]
    word2e_tok_idx=tokenizer.encode("[WORD2E]")[1]

    tokenized = tokenizer(dataset_text1,
                          dataset_text2,
                          padding=True,
                          truncation=True,
                          return_tensors="pt")
    dataset['label'] = torch.tensor(dataset['answer'],dtype=int)

    #print(tokenized["input_ids"][0])

    #word_mask1=  torch.roll((tokenized["input_ids"]==word1e_tok_idx),-1,dims=1).int()
    #word_mask2=  torch.roll((tokenized["input_ids"]==word2e_tok_idx),-1,dims=1).int()
    
    word_mask1 = torch.roll((tokenized["input_ids"]==word1s_tok_idx),1,dims=1).int()
    word_mask1 = word_mask1 - (tokenized["input_ids"]==word1e_tok_idx).int()
    word_mask1 = torch.cumsum(word_mask1,dim=1)

    word_mask2 = torch.roll((tokenized["input_ids"]==word2s_tok_idx),1,dims=1).int()
    word_mask2 = word_mask2 - (tokenized["input_ids"]==word2e_tok_idx).int()
    word_mask2 = torch.cumsum(word_mask2,dim=1)
    

    #print(word_mask[0][0])
    #print(word_mask[1][0])

    #print( torch.sum(torch.roll((tokenized["input_ids"]==worde_tok_idx),-2,dims=1).int()-(tokenized["input_ids"]==words_tok_idx).int()) )

    return dataset, tokenized, word_mask1, word_mask2


class TensorDataset(Dataset):
    def __init__(self, tokenized_dataset, word_mask1, word_mask2, labels):
        self.tokenized_dataset = tokenized_dataset
        self.word_mask1= word_mask1
        self.word_mask2= word_mask2
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.tokenized_dataset.items()}
        item["word_mask1"]=self.word_mask1[idx]
        item["word_mask2"]=self.word_mask2[idx]
        label = self.labels[idx]
        return item, label

    def __len__(self):
        return len(self.labels)

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model_type = "Roberta"
size = 'large'
model_name = f"klue/roberta-{size}"
tokenizer = AutoTokenizer.from_pretrained(model_name)

new_special_tokens=["[WORD1S]","[WORD1E]","[WORD2S]","[WORD2E]"]
tokenizer.add_special_tokens({"additional_special_tokens":new_special_tokens})
print(f"added {len(new_special_tokens)} new special token")

print(len(tokenizer))

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

added 4 new special token
32004


In [8]:

base_path = cur_dir+'data'

train_dataset, train_tokenized, train_word_mask1, train_word_mask2 = load_data(os.path.join(base_path, 'NIKL_SKT_WiC_Train.tsv'),
                                           tokenizer)
val_dataset, val_tokenized, val_word_mask1, val_word_mask2 = load_data(os.path.join(base_path, 'NIKL_SKT_WiC_Dev.tsv'), tokenizer)

train_dataset = TensorDataset(train_tokenized, train_word_mask1, train_word_mask2, train_dataset['label'])
val_dataset = TensorDataset(val_tokenized, val_word_mask1, val_word_mask2, val_dataset['label'])

if size == 'base':
    batch_size = 16
else:
    batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [9]:
# 데이터 예시
tokenizer.decode(train_tokenized['input_ids'][0])

'[CLS] 그의 죽음은 타살로 [WORD1S] 단정 [WORD1E] 이 되었다. [SEP] [WORD2S] 단정 [WORD2E] 이 된 교실은 정돈되어 있다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [10]:
# 데이터 예시2, 동형이의어 마스크
train_word_mask1[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])

### Load pretrained model
- 모델은 KLUE-RoBERTa를 사용하였습니다. (https://github.com/KLUE-benchmark/KLUE) 
- 동형이의어 마스크를 통과한 결과를 classifier에 넣고 구분하였습니다.

### 모델 수정 시도
- 1. **1 layer classifier**
- 2. 2 layer classifier
- 3. 2 layer classifier with relu
- 4. 2 layer classifier with tanh

In [11]:
class Roberta(RobertaPreTrainedModel):
    def __init__(self, config, model_name):
        super(Roberta, self).__init__(config)
        self.roberta = RobertaModel.from_pretrained(model_name, config=RobertaConfig.from_pretrained(model_name))
        self.roberta.resize_token_embeddings(config.new_tok_size)
        self.hdim = config.hidden_size
        self.nclass = config.nclass
        self.classifier = nn.Linear(self.hdim*2, self.nclass)
        #self.classifier = nn.Linear(self.hdim*2, self.hdim)
        #self.activation = nn.ReLU()
        #self.activation = nn.Tanh()
        #self.classifier2 = nn.Linear(self.hdim, self.nclass)

    def forward(self, input_ids, attention_mask, word_mask1, word_mask2 , **kwargs):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)[0]
        #(batch_size, sequence_length, hidden_size)

        #word_mask  ( 2, batch , seq_len)

        #word_mask[0] (batch, seq_len)  [[0,0,0,0,0,1,0,0,0,0],[...],..]
        #word_mask[0].unsqueeze(2) (batch, seq_len,1) 
        
        #torch.sum( word_mask[0].unsqueeze(2) * outputs , dim=1 )   # (batch,  hidden_size)
        #word_mask[1].unsqueeze(2) * outputs
        
        h = torch.cat( [torch.sum( word_mask1.unsqueeze(2) * outputs , dim=1 ),torch.sum( word_mask2.unsqueeze(2) * outputs , dim=1 )], dim=1)
        # (batch, hidden_size*2)


        logits = self.classifier(h)

        #h1 = self.classifier(h)
        #h1_a = self.activation(h1)
        #logits = self.classifier2(h1_a)

        return logits


config = RobertaConfig.from_pretrained(model_name)
config.nclass = 2
config.new_tok_size = len(tokenizer)
print(config)

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "nclass": 2,
  "new_tok_size": 32004,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.12.5",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}



In [12]:
def train_epoch(epoch, model, train_loader, optimizer, scheduler):
    model.train()
    total_loss = 0
    cor = 0
    n_sample = 0
    s = time.time()
    criterion = nn.CrossEntropyLoss()

    for data, target in tqdm(train_loader):
        item = {key: val.to(device) for key, val in data.items()}
        target = target.to(device)

        logits = model(**item)
        loss = criterion(logits, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        with torch.no_grad():
            preds = torch.argmax(logits, dim=-1)

        total_loss += loss.item()
        cor += (preds == target).sum().item()
        n_sample += len(target)

        print(f"{cor}/{n_sample}", end='\r')

    loss_avg = total_loss / n_sample
    acc = cor / n_sample
    print(
        f"[Epoch {epoch}] Train loss: {loss_avg:.3f}, acc: {acc*100:.2f}, time: {time.time()-s:.1f}s"
    )
    return acc


def validate(epoch, model, val_loader, verbose=True):
    model.eval()
    total_loss = 0
    cor = 0
    n_sample = 0
    criterion = nn.CrossEntropyLoss()
    pred_all = []
    
    with torch.no_grad():
        for data, target in val_loader:
            item = {key: val.to(device) for key, val in data.items()}
            target = target.to(device)

            logits = model(**item)
            loss = criterion(logits, target)
            preds = torch.argmax(logits, dim=-1)
            pred_all.append(preds)

            total_loss += loss.item()
            cor += (preds == target).sum().item()
            n_sample += len(target)

    loss_avg = total_loss / n_sample
    acc = cor / n_sample
    pred_all = torch.cat(pred_all)
    
    if verbose:
        print(f"[Epoch {epoch}] Valid loss: {loss_avg:.3f}, acc: {acc*100:.2f}")
    return acc, pred_all


def train(idx, num_epochs, lr, train_loader, val_loader, tokenizer):
    print(f"Start trining {idx}th model")
    model = Roberta(config, model_name).to(device)


    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = transformers.get_scheduler("linear",
                                           optimizer=optimizer,
                                           num_warmup_steps=num_epochs * len(train_loader) // 10,
                                           num_training_steps=num_epochs * len(train_loader))
    best_acc = 0
    for epoch in range(num_epochs):
        train_acc = train_epoch(epoch, model, train_loader, optimizer, scheduler)
        val_acc, _ = validate(epoch, model, val_loader)
        if val_acc > best_acc:
            best_acc = val_acc

            model_to_save = model.module if hasattr(model, "module") else model

            if not os.path.exists( os.path.join(save_dir, f'layer2_relu_{idx}') ):
                os.makedirs( os.path.join(save_dir, f'layer2_relu_{idx}') )

            model_to_save.save_pretrained(os.path.join(save_dir, f'layer2_relu_{idx}'))
            
    print(f"Training finish! Best validation accuracy: {best_acc*100:.2f}\n")

In [13]:
lr = 8e-6
num_epochs = 10

##  실행 결과
 - 각 모델은 대략 91%~92% 정도의 결과를 보임
 - 6번 실행. 이후 앙상블 적용

In [14]:
for i in range(ensen_num):
    train(i, num_epochs, lr, train_loader, val_loader, tokenizer)

Start trining 0th model


Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

  0%|          | 0/1937 [00:00<?, ?it/s]

[Epoch 0] Train loss: 0.127, acc: 73.70, time: 429.8s
[Epoch 0] Valid loss: 0.084, acc: 87.82


  0%|          | 0/1937 [00:00<?, ?it/s]

...

[Epoch 7] Train loss: 0.001, acc: 99.83, time: 434.0s
[Epoch 7] Valid loss: 0.120, acc: 91.60


  0%|          | 0/1937 [00:00<?, ?it/s]

[Epoch 8] Train loss: 0.001, acc: 99.94, time: 433.5s
[Epoch 8] Valid loss: 0.121, acc: 91.51


  0%|          | 0/1937 [00:00<?, ?it/s]

[Epoch 9] Train loss: 0.000, acc: 99.99, time: 433.1s
[Epoch 9] Valid loss: 0.126, acc: 91.68
Training finish! Best validation accuracy: 91.68



In [15]:
def train_from_my_model(idx, num_epochs, lr, train_loader, val_loader, tokenizer,model_saved_name):
    print(f"Start trining on {model_saved_name} directory ")
    model = Roberta.from_pretrained(os.path.join(save_dir, model_saved_name), model_name).to(device)

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = transformers.get_scheduler("linear",
                                           optimizer=optimizer,
                                           num_warmup_steps=num_epochs * len(train_loader) // 10,
                                           num_training_steps=num_epochs * len(train_loader))
    best_acc = 0
    for epoch in range(num_epochs):
        train_acc = train_epoch(epoch, model, train_loader, optimizer, scheduler)
        val_acc, _ = validate(epoch, model, val_loader)
        if val_acc > best_acc:
            best_acc = val_acc

            model_to_save = model.module if hasattr(model, "module") else model

            if not os.path.exists( os.path.join(save_dir, f'{idx}') ):
                os.makedirs( os.path.join(save_dir, f'{idx}') )

            model_to_save.save_pretrained(os.path.join(save_dir, f'{idx}'))
            
    print(f"Training finish! Best validation accuracy: {best_acc*100:.2f}\n")


#train_from_my_model(0, num_epochs, lr, train_loader, val_loader, tokenizer,"layer2_relu_0_epoch5_91.25_seed18")

## Test models (validation: 93.31%)
- 위의 6개의 모델을 앙상블 적용하여 최종모델 만들기 
- 정확도가 87%이하인 모델은 제외

In [16]:
"""

testing

"""




'\n\ntesting\n\n'

In [17]:
def validate_tqdm(epoch, model, val_loader, verbose=True):
    model.eval()
    total_loss = 0
    cor = 0
    n_sample = 0
    criterion = nn.CrossEntropyLoss()
    pred_all = []
    
    with torch.no_grad():
        for data, target in tqdm(val_loader):
            item = {key: val.to(device) for key, val in data.items()}
            target = target.to(device)

            logits = model(**item)
            loss = criterion(logits, target)
            preds = torch.argmax(logits, dim=-1)
            pred_all.append(preds)

            total_loss += loss.item()
            cor += (preds == target).sum().item()
            n_sample += len(target)

    loss_avg = total_loss / n_sample
    acc = cor / n_sample
    pred_all = torch.cat(pred_all)
    
    if verbose:
        print(f"[Epoch {epoch}] Valid loss: {loss_avg:.3f}, acc: {acc*100:.2f}")
    return acc, pred_all

def validate_mymodel(val_loader, answer, model_saved_name):
    model = Roberta.from_pretrained(os.path.join(save_dir, model_saved_name), model_name)
    model.to(device)
    acc, pred_all = validate_tqdm('best', model, val_loader, verbose=False)
    print(f"Load {idx}th model (acc: {acc*100:.2f})")

answer = torch.tensor(val_dataset.labels)
#validate_mymodel(val_loader, answer, "0_91.51_6epoch_seed12")

In [20]:
def validate_ensemble(val_loader, answer, idx_max=10):
    pred_ensemble = []
    for idx in range(idx_max):
        model = Roberta.from_pretrained(os.path.join(save_dir, f'layer2_relu_{idx}'), model_name)
        model.to(device)
        acc, pred_all = validate('best', model, val_loader, verbose=False)
        print(f"Load {idx}th model (acc: {acc*100:.2f})")
        if acc >= 0.87:
            pred_ensemble.append(pred_all)
        
    pred_ensemble = torch.stack(pred_ensemble, dim=-1).float()
    pred_ensemble = (pred_ensemble.mean(-1) >= 0.5).long().to(answer.device)
    acc_ensemble = (pred_ensemble == answer).sum() / len(answer)
    print(f"\nEnsemble accuracy: {acc_ensemble*100:.2f}")

In [21]:
answer = torch.tensor(val_dataset.labels)

validate_ensemble(val_loader, answer, idx_max=ensen_num)

Load 0th model (acc: 92.28)
Load 1th model (acc: 92.54)
Load 2th model (acc: 92.11)
Load 3th model (acc: 92.20)
Load 4th model (acc: 92.28)
Load 5th model (acc: 91.68)

Ensemble accuracy: 93.31
